---

In [1]:
# !pip install psycopg2-binary
# import libraries
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

##### You've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product. 

In [2]:
query = ''' SELECT * FROM books '''

In [3]:
result = pd.io.sql.read_sql(query, con = engine)
display(result.sample(5))

,book_id,author_id,title,num_pages,publication_date,publisher_id
411,412,390,Lucy Sullivan Is Getting Married,640,2007-01-23,333
665,666,530,The Burden of Proof (Kindle County Legal Thril...,608,2000-12-01,116
124,125,9,Brave New World / Brave New World Revisited,340,2005-07-05,130
518,519,139,Pope Joan,422,2009-06-09,34
218,219,108,Endymion (Hyperion Cantos #3),563,1996-12-01,40


In [4]:
query = ''' SELECT * FROM authors '''

In [5]:
result = pd.io.sql.read_sql(query, con = engine)
display(result.sample(5))

,author_id,author
104,105,Dale Carnegie
479,480,Philip Roth
91,92,Christine Feehan
388,389,Marguerite Henry/Wesley Dennis
314,315,Jon Stone/Michael J. Smollin


In [79]:
query = ''' SELECT * FROM ratings '''

In [80]:
result = pd.io.sql.read_sql(query, con = engine)
display(result)

,count
0,6456


In [77]:
query = ''' SELECT * FROM reviews '''

In [78]:
result = pd.io.sql.read_sql(query, con = engine)
display(result)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


In [10]:
query = ''' SELECT * FROM publishers '''

In [11]:
result = pd.io.sql.read_sql(query, con = engine)
display(result.sample(5))

,publisher_id,publisher
309,310,Vintage Books
125,126,Harper
112,113,Golden/Disney
308,309,Vintage
192,193,No Exit Press


### Task

##### Find the number of books released after January 1, 2000.

In [12]:
query = ''' SELECT COUNT(book_id)
FROM books 
WHERE publication_date > '2000-01-01' '''

In [13]:
result = pd.io.sql.read_sql(query, con = engine)
display(result)

,count
0,819


##### Find the number of user reviews and the average rating for each book.

In [14]:
query = ''' SELECT COUNT(DISTINCT review_id) as Num_of_Reviews, 
                   ROUND(AVG(rating),2) as avg_rating, 
                   rat.book_id
            FROM ratings as rat join reviews as rev
            ON rev.book_id = rat.book_id 
            GROUP BY rat.book_id '''

In [15]:
result = pd.io.sql.read_sql(query, con = engine)
display(result.head(5))

,num_of_reviews,avg_rating,book_id
0,2,3.67,1
1,1,2.50,2
2,3,4.67,3
3,2,4.50,4
4,4,4.00,5


##### Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).

In [16]:
 query = ''' SELECT publisher, books_num
 FROM publishers p JOIN
(SELECT publisher_id, COUNT(book_id) AS books_num 
FROM books
WHERE num_pages > 50
GROUP BY publisher_id
ORDER BY books_num DESC LIMIT 1) b
on b.publisher_id = p.publisher_id
'''   

In [17]:
result = pd.io.sql.read_sql(query, con = engine)
display(result)

,publisher,books_num
0,Penguin Books,42


Identify the author with the highest average book rating (look only at books with at least 50 ratings).

In [18]:
query = ''' SELECT author, avg_highest_rating
FROM authors a JOIN 
(SELECT AVG(rating) AS avg_highest_rating, author_id
FROM ratings r JOIN books b
on b.book_id = r.book_id
WHERE num_pages > 50
GROUP BY author_id
ORDER BY avg_highest_rating DESC LIMIT 1) sub
on a.author_id = sub.author_id
'''   

In [19]:
result = pd.io.sql.read_sql(query, con = engine)
display(result)

,author,avg_highest_rating
0,Barbara Taylor Bradford,5.0


This is the right query, check it out again please

##### Find the average number of text reviews among users who rated more than 50 books.

In [106]:
query = ''' SELECT ROUND(AVG(sub.text_num),3) AS avg_num_of_text_reviews
            FROM (SELECT COUNT(DISTINCT rev.text) AS text_num, 
                  COUNT(DISTINCT rat.book_id) as rated_books, rev.username 
                  FROM reviews rev JOIN ratings rat
                  ON rev.username = rat.username
                  GROUP BY rev.username) sub
                  WHERE rated_books > 50
                  '''  

In [107]:
result = pd.io.sql.read_sql(query, con = engine)
display(result)

,avg_num_of_text_reviews
0,24.333


Was on the rush sorry, anyway I don't think there is same review text only book_id  

##### Conclusions 

After running the queries I found out that 819 books released after January 1, 2000 and the, highest average user rating is 5.
Penguin Books is the publisher that released the greatest number of books with more than 50 pages (waw 42 books).
The author with the highest average book rating is Barbara Taylor Bradford, the average number of text reviews among users who rated more than 50 books
is 24.
I hope that I am done here. Thank you!